In [ ]:
# Steps to install
# 1. pip install sqlalchemy-bigquery google-cloud-bigquery-storage pyarrow
# 2. Copy the credentials file to wherever you set BIGQUERY_CREDENTIALS_PATH to

In [1]:
import json
import os
import pandas
import pyarrow

import sys
from os import path
import numpy

from dotenv import load_dotenv
from sqlalchemy import create_engine


load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')

In [2]:
sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
import create_queries
%load_ext autoreload
%autoreload 2

In [3]:
COMPANY_NAME = 'PL'
COMPANY_IDENTIFIER = 'PL'
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'

INVENTORY_DATE = '10/15/2021'

In [4]:
company_incoming_transfer_packages_query = create_queries.create_company_incoming_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_outgoing_transfer_packages_query = create_queries.create_company_outgoing_transfer_packages_query(COMPANY_IDENTIFIER, TRANSFER_PACKAGES_START_DATE)
company_sales_transactions_query = create_queries.create_company_sales_transactions_query(COMPANY_IDENTIFIER, SALES_TRANSACTIONS_START_DATE)
company_inventory_packages_query = create_queries.create_company_inventory_packages_query(COMPANY_IDENTIFIER)

engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))
company_incoming_transfer_packages_dataframe = pandas.read_sql_query(company_incoming_transfer_packages_query, engine)
company_outgoing_transfer_packages_dataframe = pandas.read_sql_query(company_outgoing_transfer_packages_query, engine)
company_sales_transactions_dataframe = pandas.read_sql_query(company_sales_transactions_query, engine)
company_inventory_packages_dataframe = pandas.read_sql_query(company_inventory_packages_query, engine)

In [7]:
company_incoming_transfer_packages_dataframe.head(2)

,package_row_id,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,...,product_category_name,product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,item_unit_weight,item_unit_weight_unit_of_measure_name
0,30b27089-a398-4bfb-93a6-eae9edd48a2e,INCOMING_FROM_VENDOR,C10-0000241-LIC,0002575795,2021-10-14,2021-10-15 19:15:31+00:00,C11-0000815-LIC,SAN JOSE WELLNESS,C10-0000241-LIC,"VMK, INC.",...,Extract (weight - each),MAC1 (1 GRAM),passed,0.02,2.0,Each,2.0,Each,1.0,Grams
1,a3a637bd-f430-4003-9ee3-6cf32a197c9e,INCOMING_FROM_VENDOR,C10-0000241-LIC,0002574845,2021-10-14,2021-10-14 23:32:35+00:00,C12-0000052-LIC,"ZENGANIC, INC.",C10-0000241-LIC,"VMK, INC.",...,Extract (weight - each),Community Papaya Rosin - 1g,passed,250.00,10.0,Each,10.0,Each,1.0,Grams


In [6]:
company_incoming_transfer_packages_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 28 columns):
 #   Column                                 Non-Null Count  Dtype              
---  ------                                 --------------  -----              
 0   package_row_id                         251 non-null    object             
 1   delivery_type                          251 non-null    object             
 2   license_number                         251 non-null    object             
 3   manifest_number                        251 non-null    object             
 4   created_date                           251 non-null    object             
 5   received_datetime                      246 non-null    datetime64[ns, UTC]
 6   shipper_facility_license_number        251 non-null    object             
 7   shipper_facility_name                  251 non-null    object             
 8   recipient_facility_license_number      251 non-null    object             
 9   recipient_

In [9]:
company_incoming_transfer_packages_dataframe['year_month'] = company_incoming_transfer_packages_dataframe['received_datetime'].dt.strftime("%Y-%m")

In [10]:
company_incoming_transfer_packages_dataframe['per_unit'] = company_incoming_transfer_packages_dataframe['shipper_wholesale_price'] / company_incoming_transfer_packages_dataframe['shipped_unit_of_measure']

TypeError: unsupported operand type(s) for /: 'float' and 'str'

In [58]:
incoming_transfer_simp = company_incoming_transfer_packages_dataframe[[
'license_number',
'created_date',
'package_id',
'product_category_name',
'product_name',
'shipper_wholesale_price',
'shipped_quantity',
'shipped_unit_of_measure',
'item_unit_weight',
'item_unit_weight_unit_of_measure_name']]

In [30]:
# incoming_transfer_simp

In [59]:
outgoing_transfer_simp = company_outgoing_transfer_packages_dataframe[[
'license_number',
'created_date',
'package_id',
'product_category_name',
'product_name',
'shipper_wholesale_price',
'shipped_quantity',
'shipped_unit_of_measure',
'item_unit_weight',
'item_unit_weight_unit_of_measure_name']]

In [53]:
company_sales_transactions_dataframe.to_csv('hpcc_all_sales_10_13_21.csv')

In [11]:
company_sales_transactions_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 971609 entries, 0 to 971608
Data columns (total 16 columns):
 #   Column                    Non-Null Count   Dtype              
---  ------                    --------------   -----              
 0   id                        971609 non-null  object             
 1   license_number            971609 non-null  object             
 2   receipt_number            971609 non-null  object             
 3   rt_type                   971609 non-null  object             
 4   sales_customer_type       971609 non-null  object             
 5   sales_datetime            971609 non-null  datetime64[ns, UTC]
 6   total_packages            971609 non-null  int64              
 7   rt_total_price            971609 non-null  float64            
 8   tx_type                   971609 non-null  object             
 9   tx_package_id             971609 non-null  object             
 10  tx_package_label          971609 non-null  object             
 11  

In [60]:
sales_simp = company_sales_transactions_dataframe[[
'license_number',
'sales_datetime',
'tx_package_id',
'tx_product_name',
'tx_product_category_name',
'tx_unit_of_measure',
'tx_quantity_sold',
'tx_total_price']]

In [61]:
inventory_simp = company_inventory_packages_dataframe[[
'license_number',
'package_id',
'packaged_date',
'product_name',
'product_category_name',
'quantity',
'unit_of_measure'
]]

In [62]:
sales_simp['year_month'] = sales_simp['sales_datetime'].dt.strftime("%Y-%m")

C:\Users\Rachel\GitHub\venvs\bespoke\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [40]:
sales_simp.head(2)

,license_number,sales_datetime,tx_package_id,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,month_year
0,C10-0000064-LIC,2021-10-12 20:58:15+00:00,18426625,PR THCa-Infused Preroll Lemon Crasher,Pre-Roll Infused,Each,1.0,14.71,10-2021
1,C10-0000064-LIC,2021-10-12 20:57:35+00:00,18551375,STIIIZY - Grape Pie Live Resin Pod - 1g,Vape Cartridge (weight - each),Each,1.0,54.25,10-2021


In [43]:
pandas.options.display.float_format = '{:,.2f}'.format

In [64]:
sales_simp.groupby(by=['year_month'], as_index=False).count()

,year_month,license_number,sales_datetime,tx_package_id,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price
0,2020-03,1304,1304,1304,1304,1304,1304,1304,1304
1,2020-04,2960,2960,2960,2960,2960,2960,2960,2960
2,2020-05,4228,4228,4228,4228,4228,4228,4228,4228
3,2020-06,4575,4575,4575,4575,4575,4575,4575,4575
4,2020-07,5207,5207,5207,5207,5207,5207,5207,5207
5,2020-08,5567,5567,5567,5567,5567,5567,5567,5567
6,2020-09,6034,6034,6034,6034,6034,6034,6034,6034
7,2020-10,6322,6322,6322,6322,6322,6322,6322,6322
8,2020-11,6312,6312,6312,6312,6312,6312,6312,6312
9,2020-12,4741,6520,6520,6520,6520,6520,6520,6520


In [51]:
sales_drop = sales_simp.drop(['sales_datetime', 'month_year'], axis=1)

In [52]:
sales_simp.to_csv('./hpcc_sales_2021_10_13.csv')

In [ ]:
# INVENTORY_DATES = [
#     '09/30/2020',
#     '10/31/2020',
#     '11/30/2020',
#     '12/31/2020',
#     '01/31/2021',
#     '02/28/2021',
#     '03/31/2021',
#     '04/30/2021',
#     '05/31/2021',
#     '06/30/2021',
#     '07/31/2021',
#     '08/31/2021',
#     '09/30/2021',
# ]

In [12]:
# %autoreload 2

# sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts/analysis")))
# sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

# from util import active_inventory_util as util

In [13]:
# d = util.Download()
# d.download_dataframes(
#     incoming_transfer_packages_dataframe=company_incoming_transfer_packages_dataframe,
#     outgoing_transfer_packages_dataframe=company_outgoing_transfer_packages_dataframe,
#     sales_transactions_dataframe=company_sales_transactions_dataframe,
# )

In [19]:
# q = util.Query()
# q.inventory_dates = INVENTORY_DATES
# q.company_name = COMPANY_NAME

# id_to_history = util.get_histories(d)
# util.print_counts(id_to_history)
# util.create_inventory_xlsx(id_to_history, q)

In [20]:
# computed_inventory_package_records = util.create_inventory_dataframe_by_date(id_to_history, INVENTORY_DATE)
# computed_inventory_packages_dataframe = pandas.DataFrame(
#     computed_inventory_package_records,
#     columns=[
#         'package_id',
#         'Arrived Date',
#         'Product Category',
#         'Product Name',
#         'Current Quantity',
#         'Sold Date',
#     ]
# )
# computed_inventory_packages_dataframe

In [21]:
# date_to_inventory_dataframe = {}
# for date, inventory_records in date_to_inventory_records.items():
#     date_to_inventory_dataframe[date] = pandas.DataFrame(
#         inventory_records,
#         columns=[
#             'package_id',
#             'Arrived Date',
#             'Product Category',
#             'Product Name',
#             'Current Quantity',
#             'Sold Date',
#         ]
#     )
    
# date_to_inventory_dataframe[list(date_to_inventory_dataframe.keys())[0]]

In [17]:
# raw_incoming_transfer_packages_dataframe = pandas.read_excel('data/20210930/royal_apothecary_incoming_transfer_packages_20200101_20210930.xlsx', header=0)
# len(raw_incoming_transfer_packages_dataframe.index), raw_incoming_transfer_packages_dataframe.columns

In [88]:
# incoming_transfer_packages_dataframe = raw_incoming_transfer_packages_dataframe[[
#     'date_type',
#     'transfer_row_id',
#     'delivery_row_id',
#     'package_row_id',
#     'delivery_type',
#     'manifest_number',
#     'created_date',
#     'received_datetime',
#     'shipper_facility_license_number',
#     'shipper_facility_name',
#     'recipient_facility_license_number',
#     'recipient_facility_name',
#     'shipment_type_name',
#     'shipment_transaction_type',
#     'package_id',
#     'package_label',
#     'type',
#     'product_category_name',
#     'product_name',
#     'shipper_wholesale_price',
#     'shipped_quantity',
#     'package_lab_results_status',
#     'shipment_package_state',
#     'is_testing_sample',
#     'is_trade_sample'
# ]]

In [18]:
# for date, inventory_dataframe in date_to_inventory_dataframe.items():
#     print(date)
#     print(f'# of packages in inventory: {len(inventory_dataframe.index)}')

#     inventory_with_incoming_transfer_packages_dataframe = inventory_dataframe.astype({'package_id': 'int64'}).merge(incoming_transfer_packages_dataframe, on='package_id', how='inner', suffixes=('_l', '_r'))
# #     print(f'# of packages in inventory with incoming package: {len(inventory_with_incoming_transfer_packages_dataframe.index)}')
    
#     inventory_with_cost_records = inventory_with_incoming_transfer_packages_dataframe.to_dict('record')

#     total_valuation_cost = 0
#     import math
#     for inventory_with_cost_record in inventory_with_cost_records:
#         incoming_shipped_price = inventory_with_cost_record['shipper_wholesale_price']
#         if math.isnan(incoming_shipped_price):
#             incoming_shipped_price = 0
#         incoming_shipped_quantity = inventory_with_cost_record['shipped_quantity']
#         current_quantity = inventory_with_cost_record['Current Quantity']
#         total_valuation_cost += float(current_quantity) * (incoming_shipped_price / incoming_shipped_quantity)

#     print(f'Inventory valuation (based on COST): ${total_valuation_cost}')
#     print('')

In [ ]:
# import json
# import pandas
# import numpy
# import os
# import sys
# from os import path